# IMPORTAÇÕES

In [111]:
import pandas as pd
import numpy as np

from copy import copy
from math import inf
from pandas.api.types import is_numeric_dtype
from typing import Any, Callable, Dict, List, Union

# FUNÇÕES

In [112]:
class Environment:
    def __init__(self, root, mode, source, subsource, year, month, file, extension):
        self.root = root            # Pasta raiz
        self.mode = mode            # Pasta referente ao processamento de dados ("raw" [não processado] vs "cooked" [processado])
        self.source = source        # Pasta referente à fonte de dados
        self.subsource = subsource  # Pasta referente à fonte de dados
        self.year = year            # Ano
        self.month = month          # Mês
        self.file = file            # Nome do arquivo (sem extensão)
        self.extension = extension  # Extensão do arquivo

    def get_file_path(self):
      """
      Retorna caminho do arquivo.
      :return: Caminho do arquivo.
      """
      file_name = f"{self.file.format(year=self.year, month=self.month)}{self.extension}"
      parts = [
          self.root,
          self.mode,
          self.source,
          self.subsource,
          self.year,
          file_name,
      ]
      return "/".join(parts)

    def clone(self):
      """
      Retorna uma cópia rasa (shallow copy) do objeto. Ou seja,
      as referências não serão clonadas; apenas o objeto em si.
      :return: Cópia rasa do objeto.
      """
      return copy(self)

def df_show_domain(df: pd.DataFrame, columns: List[str]) -> None:
  """
  Exibe o tipo e os valores possíveis de columns segundo a ordem da lista de
  colunas.

  :param df: DataFrame a ter os domínios exibidos.
  :param columns: Lista dos nomes das colunas a serem exibidas.
  :return: None
  """
  for col in columns:
    print(f"Coluna:   {col}")
    print(f"dtype:    {df[col].dtype}")
    print(f"Domínio:  {df[col].unique()}\n")

def df_show_null(df: pd.DataFrame) -> None:
  """
  Exibe o número de valores nulos por coluna.

  :param df: DataFrame a ter os domínios exibidos.
  :return: None
  """
  print("# Número de valores nulos por coluna:")
  display(df.isnull().sum())

def df_subs_noise(df: pd.DataFrame, dict_dom: Dict[str, List[Any]]) -> pd.DataFrame:
    """
    Substitui valores inválidos em um DataFrame com base em um dicionário de domínios,
    utilizando pd.NA para representar valores ausentes.

    :param df: DataFrame a ser processado.
    :param dict_dom: Dicionário em que cada chave é o nome de uma coluna e o valor é uma lista contendo:
                     - Para tipos numéricos: [tipo, min, max] (inclui extremos)
                     - Para outros tipos: [tipo, valor1, valor2, ...]
    :return: Novo DataFrame com valores inválidos substituídos por pd.NA.
    :rtype: pd.DataFrame
    """
    # CONSTANTES AUXILIARES
    TYP_IDX = 0 # Índice do tipo

    # Apenas para tipos numéricos:
    MIN_IDX = 1     # Índice do elemento mínimo
    MAX_IDX = 2     # Índice do elemento máximo
    NUM_LEN = 3     # Tamanho adequado do vetor de domínio
    EPSILON = 1E-10 # Tolerância (para operações com pontos flutuantes)

    df = df.copy()

    for col, dom in dict_dom.items():
        expected_type = dom[TYP_IDX]

        if is_numeric_dtype(df[col]): # NUMÉRICO
            if len(dom) != NUM_LEN:
                raise ValueError(f"Para a coluna '{col}', dom_dict deve conter exatamente três elementos: [tipo, min, max].")
            min_val, max_val = dom[MIN_IDX], dom[MAX_IDX]

            def validate_numeric(val): # Função auxiliar de validação numérica
                try:
                    num = expected_type(val) # Tenta converter
                    if min_val - EPSILON <= num <= max_val + EPSILON: # Verifica se está contido no intervalo adequado considerando a tolerância
                        return num
                    else:
                        return pd.NA

                except (ValueError, TypeError):
                    return pd.NA
            df[col] = df[col].apply(validate_numeric) # Valida coluna elemento a elemento

        else: # NÃO NUMÉRICO
            valid_values = set(dom[1:]) # Obtém domínio e evita duplicatas com o uso de set
            df[col] = df[col].apply(lambda x: x if x in valid_values else pd.NA) # Substitui ruído por pd.NA

    # Converter os tipos de dados para os tipos "nullable" do Pandas
    df = df.convert_dtypes()
    return df

def new_name(col_name: str, dict_agg: Dict[str, str], dict_cat: Dict[str, str], dict_radix: Dict[str, str]) -> str:
  """
  Gera um novo nome com base nos atributos anteriores. Se a coluna não estiver no
  dicionário de categorias, assume-se a categoria padrão ("geral"). E, se não
  estiver em no dicionário de raízes, assume-se o nome da coluna como raiz.

  :param col_name: Nome da coluna a ter seu novo nome calculado.
  :param dict_agg: Dicionário usado com groupby durante a agregação.
  :param dict_cat: Dicionário em que cada chave é o nome da coluna, e o valor
                   é sua categoria.
  :param dict_radix: Dicionário que mapeia colunas em raízes léxicas (parte do meio
                do novo nome de coluna).
  :return: Novo nome da coluna.
  """
  # Constantes auxiliares
  DEFAULT_PREFIX = ""

  # Indica categoria
  PREFIX = {
    "carbono":  "car",
    "clima":    "cli",
    "queimada": "que"
  }

  # Indica função de agregação do pré-processamento
  SUFFIX = {
    "count":  "qtd", # Quantidade
    "mean":   "med"  # Média
  }

  # Obtém raiz e afixos do novo nome
  category = dict_cat.get(col_name)
  prefix = f"{PREFIX.get(category, DEFAULT_PREFIX)}_"
  radix = dict_radix.get(col_name, col_name)
  agg = dict_agg.get(col_name, "")
  suffix = f"_{SUFFIX.get(agg, '')}" if agg else ""

  return f"{prefix}{radix}{suffix}"

def dict_new_name(columns: List[str], dict_agg: Dict[str, str], dict_cat: Dict[str, str], dict_radix: Dict[str, str]) -> Dict[str, str]:
  """
  Gera um dicionário de novos nomes para as colunas. Se a coluna não estiver no
  dicionário de categorias, assume-se a categoria padrão ("geral"). E, se não
  estiver em no dicionário de raízes, assume-se o nome da coluna como raiz.

  :param columns: Coluna a terem novo nome.
  :param dict_cat: Dicionário em que cada chave é o nome da coluna, e o valor
                   é sua categoria.
  :param radix: Dicionário que mapeia colunas em raízes léxicas (parte do meio
                do novo nome de coluna).
  :return: Dicionário com pares [coluna, novo-nome].
  """
  return { col: new_name(col, dict_agg, dict_cat, dict_radix) for col in columns }

def df_new_name(df: pd.DataFrame, columns: List[str], dict_agg: Dict[str, str], dict_cat: Dict[str, str], dict_radix: Dict[str, str]) -> pd.DataFrame:
  """
  Gera um novo DataFrame com os novos nomes. Se a coluna não estiver no
  dicionário de categorias, assume-se a categoria padrão ("geral"). E, se não
  estiver em no dicionário de raízes, assume-se o nome da coluna como raiz.

  :param df: DataFrame a ter seus nomes alterados.
  :param columns: Colunas a terem novo nome.
  :param dict_cat: Dicionário em que cada chave é o nome da coluna, e o valor
                   é sua categoria.
  :param radix: Dicionário que mapeia colunas em raízes léxicas (parte do meio
                do novo nome de coluna).
  :return: Dicionário com pares [coluna, novo-nome].
  """
  dict_names = dict_new_name(columns, dict_agg, dict_cat, dict_radix)
  return df.rename(columns=dict_names)

def df_new_cols(
    df: pd.DataFrame,
    aggregator: str,
    new_cols: List[str],
    new_funcs: List[Callable],
    new_types: List[Union[str, type]]
) -> pd.DataFrame:
    """
    Adiciona novas colunas derivadas a partir de uma coluna agregadora, aplicando
    funções de transformação e tipos específicos.

    :param df: DataFrame original que será copiado e modificado.
    :param aggregator: Nome da coluna cujos valores serão usados como base para as transformações.
    :param new_cols: Lista com os nomes das novas colunas a serem adicionadas.
    :param new_funcs: Lista de funções a serem aplicadas aos valores da coluna agregadora.
    :param new_types: Lista de tipos que serão atribuídos às novas colunas após transformação.
    :return: Um novo DataFrame com as colunas adicionais.
    """
    if not (len(new_cols) == len(new_funcs) == len(new_types)):
      raise ValueError("Listas new_cols, new_funcs e new_types devem ter o mesmo comprimento.")

    df = df.copy()
    for i in range(len(new_cols)):
        df[new_cols[i]] = df[aggregator].apply(new_funcs[i]).astype(new_types[i])
    return df

def int_to_str_with_zeros(n: int, size: int) -> str:
    return str(n).zfill(size)

def perror(e: Exception):
    print(f"Handled error of type: {type(e).__name__}")
    print(f"Error message: {e}")

# SETUP DE AMBIENTE

In [113]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# OBTENÇÃO DOS DATASETS

In [114]:
# Caminho para o arquivo de entrada
ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets"
mode = "raw"                # Pasta de dados brutos
SRC = "TerraBrasilis"       # Fonte da pesquisa
SUB_SRC = "focos_mensal_br" # Sub-fonte da pesquisa
year = "2023"
month = "01"
FILE_FORMAT = "focos_mensal_br_{year}{month}"
EXT = ".csv"

# Instancia classe de ambiente
env_in = Environment(ROOT, mode, SRC, SUB_SRC, year, month, FILE_FORMAT, EXT)

# Cria DataFrame e exibe resultado
df_raw = pd.read_csv(env_in.get_file_path())
display(df_raw)

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
0,8c3be1cf-55a4-3d95-9e1d-6edbbb9985c9,-13.25160,-45.55090,2023-01-01 01:13:30,METOP-B,CORRENTINA,BAHIA,Brasil,2909307,29,33,NaN,NaN,NaN,Cerrado,NaN
1,61d686ec-fc2f-3c5c-8cf1-ece94ec2feac,-13.24710,-45.53360,2023-01-01 01:13:30,METOP-B,CORRENTINA,BAHIA,Brasil,2909307,29,33,NaN,NaN,NaN,Cerrado,NaN
2,1281243c-38f7-30db-92d8-31356fb06af4,-13.26891,-45.56372,2023-01-01 01:28:00,TERRA_M-M,CORRENTINA,BAHIA,Brasil,2909307,29,33,NaN,NaN,NaN,Cerrado,23.8
3,3ce4a9d8-6d75-3722-b6fe-7e4847893273,-13.26745,-45.55385,2023-01-01 01:28:00,TERRA_M-M,CORRENTINA,BAHIA,Brasil,2909307,29,33,NaN,NaN,NaN,Cerrado,20.0
4,682f75d4-40ef-38bf-abed-dfa1fc5b1688,-13.33733,-44.11688,2023-01-01 01:28:00,TERRA_M-M,SANTA MARIA DA VITÓRIA,BAHIA,Brasil,2928109,29,33,NaN,NaN,NaN,Cerrado,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40974,4b29cb42-1dbd-37f3-9498-a0e8f85a5d2d,-6.12100,-35.26810,2023-01-31 23:45:40,GOES-16,SÃO JOSÉ DE MIPIBU,RIO GRANDE DO NORTE,Brasil,2412203,24,33,12.0,0.48,1.00,Mata Atlântica,83.2
40975,6150ea36-d88a-394d-9921-5d78e6b1f054,-6.12130,-35.24080,2023-01-31 23:45:40,GOES-16,SÃO JOSÉ DE MIPIBU,RIO GRANDE DO NORTE,Brasil,2412203,24,33,12.0,0.20,0.78,Mata Atlântica,59.5
40976,04e25531-8c67-3124-a229-053d67f87e7b,2.94630,-60.17900,2023-01-31 23:54:49,GOES-16,BONFIM,RORAIMA,Brasil,1400159,14,33,7.0,0.00,0.26,Amazônia,101.4
40977,948c2f07-9beb-3635-8fab-dbfa1cbfca08,2.96460,-60.17870,2023-01-31 23:54:49,GOES-16,BONFIM,RORAIMA,Brasil,1400159,14,33,7.0,0.00,0.67,Amazônia,95.7


# PRÉ-PROCESSAMENTO

## ESTRUTURAÇÃO

In [115]:
# INVESTIGAÇÃO DE DOMÍNIO
df_show_domain(df_raw, df_raw.columns)
df_show_null(df_raw)

Coluna:   id
dtype:    object
Domínio:  ['8c3be1cf-55a4-3d95-9e1d-6edbbb9985c9'
 '61d686ec-fc2f-3c5c-8cf1-ece94ec2feac'
 '1281243c-38f7-30db-92d8-31356fb06af4' ...
 '04e25531-8c67-3124-a229-053d67f87e7b'
 '948c2f07-9beb-3635-8fab-dbfa1cbfca08'
 'b3091831-7acc-370e-9d55-e5c98bb2c013']

Coluna:   lat
dtype:    float64
Domínio:  [-13.2516  -13.2471  -13.26891 ...   3.0561    2.9464   -6.1019 ]

Coluna:   lon
dtype:    float64
Domínio:  [-45.5509  -45.5336  -45.56372 ... -60.1201  -60.16    -35.2703 ]

Coluna:   data_hora_gmt
dtype:    object
Domínio:  ['2023-01-01 01:13:30' '2023-01-01 01:28:00' '2023-01-01 03:29:00' ...
 '2023-01-31 23:45:40' '2023-01-31 23:54:49' '2023-01-31 23:56:05']

Coluna:   satelite
dtype:    object
Domínio:  ['METOP-B' 'TERRA_M-M' 'NPP-375D' 'NOAA-20' 'TERRA_M-T' 'GOES-16'
 'NPP-375' 'AQUA_M-T' 'NOAA-19' 'MSG-03' 'AQUA_M-M' 'NOAA-18D' 'NOAA-19D'
 'METOP-C']

Coluna:   municipio
dtype:    object
Domínio:  ['CORRENTINA' 'SANTA MARIA DA VITÓRIA' 'SERTÂNIA' ... 'CATU

,0
id,0
lat,0
lon,0
data_hora_gmt,0
satelite,0
municipio,0
estado,0
pais,0
municipio_id,0
estado_id,0


### CRITÉRIO DE REMOÇÃO

Como este *dataset* fornecerá, por estado e por mês, apenas os dados referentes a
- Número de dias sem chuva;
- Risco de fogo;
- Fire Radiance Power (FRP);
- Quantidade de focos de incêndio (obtida indiretamente por meio do agregador `count("id")` atrelado a `estado`);

não faz sentido manter todas as colunas. Por isso, mantiveram-se apenas as descritas em `KEEPING`, que ou os contêm diretamente, ou os permitem calcular. Vale ressaltar que, em qualquer caso, será mantida a coluna `id`, porque preserva a individualidade das tuplas originais, imprescindível para o devido cálculo de agregadores.

In [116]:
# REMOÇÃO DE COLUNAS DESNECESSÁRIAS
KEEPING = ["id", "estado", "numero_dias_sem_chuva", "risco_fogo", "frp"]
REMOVING = [x for x in df_raw.columns if x not in KEEPING]

# Remove colunas desnecessárias
df = df_raw.drop(columns=REMOVING)
display(df)

,id,estado,numero_dias_sem_chuva,risco_fogo,frp
0,8c3be1cf-55a4-3d95-9e1d-6edbbb9985c9,BAHIA,NaN,NaN,NaN
1,61d686ec-fc2f-3c5c-8cf1-ece94ec2feac,BAHIA,NaN,NaN,NaN
2,1281243c-38f7-30db-92d8-31356fb06af4,BAHIA,NaN,NaN,23.8
3,3ce4a9d8-6d75-3722-b6fe-7e4847893273,BAHIA,NaN,NaN,20.0
4,682f75d4-40ef-38bf-abed-dfa1fc5b1688,BAHIA,NaN,NaN,7.0
...,...,...,...,...,...
40974,4b29cb42-1dbd-37f3-9498-a0e8f85a5d2d,RIO GRANDE DO NORTE,12.0,1.00,83.2
40975,6150ea36-d88a-394d-9921-5d78e6b1f054,RIO GRANDE DO NORTE,12.0,0.78,59.5
40976,04e25531-8c67-3124-a229-053d67f87e7b,RORAIMA,7.0,0.26,101.4
40977,948c2f07-9beb-3635-8fab-dbfa1cbfca08,RORAIMA,7.0,0.67,95.7


## LIMPEZA

### DEFINIÇÃO DE DOMÍNIOS

Nesta seção, serão substituídos por `NA` (`Not Available`) aqueles dados que se configurem como ruído, isto é, aqueles que nitidamente firam o domínio de dados, como uma idade negativa ou um CPF de 13 dígitos. Neste caso, o domínio dos dados foi previamente definido no dicionário de dados do projeto.

In [117]:
# DEFINIÇÃO DE DOMÍNIOS
dict_dom = {
    "estado":                 [str, "ACRE", "AMAZONAS", "AMAPÁ", "MARANHÃO",
                               "MATO GROSSO", "PARÁ", "RONDÔNIA", "RORAIMA",
                               "TOCANTINS"],
    "numero_dias_sem_chuva":  [np.int32, 0, +inf],
    "risco_fogo":             [np.float64, 0.0, 1.0],
    "frp":                    [np.float64, 0.0, +inf]
}

# Substitui ruídos por pd.NA
df_noiseless = df_subs_noise(df, dict_dom)

# Converte nomes extensos em siglas, para seguir o domínio
dict_states = {
    "ACRE":         "AC",
    "AMAZONAS":     "AM",
    "AMAPÁ":        "AP",
    "MARANHÃO":     "MA",
    "MATO GROSSO":  "MT",
    "PARÁ":         "PA",
    "RONDÔNIA":     "RO",
    "RORAIMA":     "RR",
    "TOCANTINS":    "TO"
}
# Mapeia em pd.NA se for ruído
df_noiseless["estado"] = df_noiseless["estado"].map(lambda x: dict_states.get(x, pd.NA)).astype("string")

# Exibe resultado
display(df_noiseless)
df_show_domain(df_noiseless, df_noiseless.columns)

,id,estado,numero_dias_sem_chuva,risco_fogo,frp
0,8c3be1cf-55a4-3d95-9e1d-6edbbb9985c9,<NA>,<NA>,<NA>,<NA>
1,61d686ec-fc2f-3c5c-8cf1-ece94ec2feac,<NA>,<NA>,<NA>,<NA>
2,1281243c-38f7-30db-92d8-31356fb06af4,<NA>,<NA>,<NA>,23.8
3,3ce4a9d8-6d75-3722-b6fe-7e4847893273,<NA>,<NA>,<NA>,20.0
4,682f75d4-40ef-38bf-abed-dfa1fc5b1688,<NA>,<NA>,<NA>,7.0
...,...,...,...,...,...
40974,4b29cb42-1dbd-37f3-9498-a0e8f85a5d2d,<NA>,12,1.0,83.2
40975,6150ea36-d88a-394d-9921-5d78e6b1f054,<NA>,12,0.78,59.5
40976,04e25531-8c67-3124-a229-053d67f87e7b,RR,7,0.26,101.4
40977,948c2f07-9beb-3635-8fab-dbfa1cbfca08,RR,7,0.67,95.7


Coluna:   id
dtype:    string
Domínio:  <StringArray>
['8c3be1cf-55a4-3d95-9e1d-6edbbb9985c9',
 '61d686ec-fc2f-3c5c-8cf1-ece94ec2feac',
 '1281243c-38f7-30db-92d8-31356fb06af4',
 '3ce4a9d8-6d75-3722-b6fe-7e4847893273',
 '682f75d4-40ef-38bf-abed-dfa1fc5b1688',
 'e40fb522-7f54-37c2-97bd-3985c4f99544',
 'f50fc95c-4838-3b54-8fd1-cbea08337a62',
 'eb32c557-57ad-35a6-82a1-db247b93a7ad',
 '4f1c5ec2-3247-3236-bbe2-86c9469fad04',
 '41cfd015-3489-31f4-bf86-54825552d429',
 ...
 'df7f98e8-b7e9-34b6-8da2-735b8eab5bc5',
 '690fd0b2-1ad5-3b41-8cc0-7214e5d36d01',
 'd85ebccd-7604-366c-9069-97687df654ec',
 '287db666-f08c-3de1-9566-82d5c0d559ae',
 '79e87b88-8187-3b04-9779-d805f307fa29',
 '4b29cb42-1dbd-37f3-9498-a0e8f85a5d2d',
 '6150ea36-d88a-394d-9921-5d78e6b1f054',
 '04e25531-8c67-3124-a229-053d67f87e7b',
 '948c2f07-9beb-3635-8fab-dbfa1cbfca08',
 'b3091831-7acc-370e-9d55-e5c98bb2c013']
Length: 40979, dtype: string

Coluna:   estado
dtype:    string
Domínio:  <StringArray>
[<NA>, 'MA', 'MT', 'PA', 'RO', 'T

## AGREGAÇÃO

In [118]:
AGGREGATOR = "estado"

dict_agg = {
    "id": "count", # Serve para cálculo de focos de incêndio
    "numero_dias_sem_chuva": "mean",
    "frp": "mean",
    "risco_fogo": "mean"
}

# Agrega colunas por estado segundo as funções especificadas
df_agg = df_noiseless.groupby(AGGREGATOR).agg(dict_agg).reset_index()
display(df_agg)
df_show_domain(df_agg, df_agg.columns)

,estado,id,numero_dias_sem_chuva,frp,risco_fogo
0,AC,92,1.09434,29.465217,0.009057
1,AM,475,1.743056,11.438316,0.004016
2,AP,34,0.857143,9.420588,0.035806
3,MA,3577,0.996765,13.518561,0.166536
4,MT,2882,1.657726,84.602393,0.027833
5,PA,4558,0.792525,13.603198,0.059526
6,RO,710,1.151361,18.95773,0.036758
7,RR,4799,5.413478,17.492165,0.335404
8,TO,621,0.642987,10.195169,0.005672


Coluna:   estado
dtype:    string
Domínio:  <StringArray>
['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']
Length: 9, dtype: string

Coluna:   id
dtype:    int64
Domínio:  [  92  475   34 3577 2882 4558  710 4799  621]

Coluna:   numero_dias_sem_chuva
dtype:    Float64
Domínio:  <FloatingArray>
[1.0943396226415094, 1.7430555555555556, 0.8571428571428571,
  0.996764917325665, 1.6577261809447559, 0.7925247902364607,
  1.151360544217687,  5.413477891156463, 0.6429872495446266]
Length: 9, dtype: Float64

Coluna:   frp
dtype:    Float64
Domínio:  <FloatingArray>
[ 29.46521739130435, 11.438315789473684,  9.420588235294119,
 13.518561030235164,  84.60239268121042, 13.603198059108955,
   18.9577304964539,  17.49216503438216, 10.195169082125604]
Length: 9, dtype: Float64

Coluna:   risco_fogo
dtype:    Float64
Domínio:  <FloatingArray>
[0.009056603773584906, 0.004015957446808511, 0.035806451612903224,
  0.16653598971722364,  0.02783273019702453,  0.05952586206896552,
  0.03675814751286449

## CRIAÇÃO DE NOVAS COLUNAS

In [119]:
NEW_COLS = ["ano", "mes"]                         # Nomes
NEW_FUNCS = [lambda agg: year, lambda agg: month] # Função de preenchimento sobre a coluna agregadora
NEW_TYPES = ["Int32", "Int32"]                    # Tipos

# Cria novas colunas
df_agg = df_new_cols(df_agg, AGGREGATOR, NEW_COLS, NEW_FUNCS, NEW_TYPES)

# Exibe resultado das novas colunas
display(df_agg)
df_show_domain(df_agg, NEW_COLS)

,estado,id,numero_dias_sem_chuva,frp,risco_fogo,ano,mes
0,AC,92,1.09434,29.465217,0.009057,2023,1
1,AM,475,1.743056,11.438316,0.004016,2023,1
2,AP,34,0.857143,9.420588,0.035806,2023,1
3,MA,3577,0.996765,13.518561,0.166536,2023,1
4,MT,2882,1.657726,84.602393,0.027833,2023,1
5,PA,4558,0.792525,13.603198,0.059526,2023,1
6,RO,710,1.151361,18.95773,0.036758,2023,1
7,RR,4799,5.413478,17.492165,0.335404,2023,1
8,TO,621,0.642987,10.195169,0.005672,2023,1


Coluna:   ano
dtype:    Int32
Domínio:  <IntegerArray>
[2023]
Length: 1, dtype: Int32

Coluna:   mes
dtype:    Int32
Domínio:  <IntegerArray>
[1]
Length: 1, dtype: Int32



## RENOMEAÇÃO DE COLUNAS

In [120]:
# Mapeamento de colunas em categorias
CATEGORIES = {
    "id": "queimada",
    "numero_dias_sem_chuva": "queimada",
    "frp": "queimada",
    "risco_fogo": "queimada"
}

# Mapeamento de mudança de raiz do nome
RADIX = {
    "id": "focos",
    "numero_dias_sem_chuva": "num_dia_sem_chuva"
}

# Altera nome das colunas
df_final = df_new_name(df_agg, df_agg.columns, dict_agg, CATEGORIES, RADIX)
df_final = df_final[sorted(df_final.columns)] # Reordena alfabeticamente
display(df_final)

,_ano,_estado,_mes,que_focos_qtd,que_frp_med,que_num_dia_sem_chuva_med,que_risco_fogo_med
0,2023,AC,1,92,29.465217,1.09434,0.009057
1,2023,AM,1,475,11.438316,1.743056,0.004016
2,2023,AP,1,34,9.420588,0.857143,0.035806
3,2023,MA,1,3577,13.518561,0.996765,0.166536
4,2023,MT,1,2882,84.602393,1.657726,0.027833
5,2023,PA,1,4558,13.603198,0.792525,0.059526
6,2023,RO,1,710,18.95773,1.151361,0.036758
7,2023,RR,1,4799,17.492165,5.413478,0.335404
8,2023,TO,1,621,10.195169,0.642987,0.005672


# SALVAMENTO

In [121]:
env_out = env_in.clone()
env_out.mode = "cooked" # Salvamento na pasta de arquivos já processados
env_out.file += f"_{env_out.mode}" # Adiciona sufixo indicador de processamento

# Salva arquivo processado
df_final.to_csv(env_out.get_file_path(), index=False, encoding="utf-8")

# REPETIÇÃO

Agora, serão repetidos os passos anteriores para todos os conjuntos de dados, uma vez que se trata de bases com a mesma estrutura. Para evitar poluições gráficas, serão omitidos os comandos que produzam alguma exibição na console.

In [122]:
def main():
  # PARÂMETROS ###############
  BEG_MONTH = 1
  FIN_MONTH = 12
  LEN_MONTH = 2
  BEG_YEAR = 2023
  FIN_YEAR = 2025
  LEN_YEAR = 4

  # AMBIENTE
  ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets"
  SRC = "TerraBrasilis"
  SUB_SRC = "focos_mensal_br"
  FILE_FORMAT = "focos_mensal_br_{year}{month}"
  EXT = ".csv"

  # REMOÇÃO DE COLUNAS
  KEEPING = ["id", "estado", "numero_dias_sem_chuva", "risco_fogo", "frp"]

  # DOMÍNIO
  DOM = {
      "estado":                 [str, "ACRE", "AMAZONAS", "AMAPÁ", "MARANHÃO",
                                "MATO GROSSO", "PARÁ", "RONDÔNIA", "RORAIMA",
                                "TOCANTINS"],
      "numero_dias_sem_chuva":  [np.int32, 0, +inf],
      "risco_fogo":             [np.float64, 0.0, 1.0],
      "frp":                    [np.float64, 0.0, +inf]
  }
  STATES = {
      "ACRE":         "AC",
      "AMAZONAS":     "AM",
      "AMAPÁ":        "AP",
      "MARANHÃO":     "MA",
      "MATO GROSSO":  "MT",
      "PARÁ":         "PA",
      "RONDÔNIA":     "RO",
      "RORAIMA":     "RR",
      "TOCANTINS":    "TO"
  }

  # AGREGAÇÃO
  AGGREGATOR = "estado"
  AGG = {
      "id": "count", # Serve para cálculo de focos de incêndio
      "numero_dias_sem_chuva": "mean",
      "frp": "mean",
      "risco_fogo": "mean"
  }

  # NOVAS COLUNAS
  NEW_COLS = ["ano", "mes"]           # Nomes
  new_funcs = [None] * len(NEW_COLS)  # Função de preenchimento sobre a coluna agregadora
  YEAR_IDX = 0
  MONTH_IDX = 1
  NEW_TYPES = ["Int32", "Int32"]      # Tipos

  # EDIÇÃO DE NOMES
  CATEGORIES = {
      "id": "queimada",
      "numero_dias_sem_chuva": "queimada",
      "frp": "queimada",
      "risco_fogo": "queimada"
  }
  RADIX = {
      "id": "focos",
      "numero_dias_sem_chuva": "num_dia_sem_chuva"
  }
  ##############################

  # Intervalo de datasets
  MONTHS = [x for x in range(BEG_MONTH, FIN_MONTH + 1)]
  YEARS = [x for x in range(BEG_YEAR, FIN_YEAR + 1)]
  df_raw = None

  # Ambiente de entrada
  env_in = Environment(ROOT, "raw", SRC, SUB_SRC, BEG_YEAR, BEG_MONTH, FILE_FORMAT, EXT)

  # Ambiente de saída
  env_out = env_in.clone()
  env_out.mode = "cooked"
  env_out.file += f"_{env_out.mode}"

  for year in YEARS:
    env_in.year = env_out.year = int_to_str_with_zeros(year, LEN_YEAR)
    new_funcs[YEAR_IDX] = lambda agg: year # Função de preenchimento sobre a coluna agregadora

    for month in MONTHS:
      env_in.month = env_out.month = int_to_str_with_zeros(month, LEN_MONTH)
      new_funcs[MONTH_IDX] = lambda agg: month # Função de preenchimento sobre a coluna agregadora

      # ESTRUTURAÇÃO
      try:
        df_raw = pd.read_csv(env_in.get_file_path())
      except (FileNotFoundError, ValueError, TypeError) as e:
        perror(e)
        continue
      df = df_raw[KEEPING]

      # LIMPEZA
      df_noiseless = df_subs_noise(df, DOM)
      df_noiseless["estado"] = df_noiseless["estado"].map(lambda x: STATES.get(x, pd.NA)).astype("string")

      # AGREGAÇÃO
      df_agg = df_noiseless.groupby(AGGREGATOR).agg(AGG).reset_index()

      # ALTERAÇÃO DE COLUNAS
      df_agg = df_new_cols(df_agg, AGGREGATOR, NEW_COLS, new_funcs, NEW_TYPES)
      df_final = df_new_name(df_agg, df_agg.columns, AGG, CATEGORIES, RADIX)
      df_final = df_final[sorted(df_final.columns)] # Reordena alfabeticamente

      # SALVAMENTO
      try:
        df_final.to_csv(env_out.get_file_path(), index=False, encoding="utf-8")
      except (FileNotFoundError, ValueError, TypeError) as e:
        perror(e)
        continue

In [123]:
# ROTINA DE PRÉ-PROCESSAMENTO
main()

Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/raw/TerraBrasilis/focos_mensal_br/2025/focos_mensal_br_202505.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/raw/TerraBrasilis/focos_mensal_br/2025/focos_mensal_br_202506.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/raw/TerraBrasilis/focos_mensal_br/2025/focos_mensal_br_202507.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/raw/TerraBrasilis/focos_mensal_br/2025/focos_mensal_br_202508.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive